In [ ]:
%cd /home/alpaco/namu/LLaMA-Adapter/llama_adapter_v2_multimodal7b

/home/alpaco/namu/LLaMA-Adapter/llama_adapter_v2_multimodal7b


In [ ]:
#!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
  Cloning https://github.com/csuhan/timm_0_3_2.git to /tmp/pip-req-build-ghq9_qab
  Running command git clone --filter=blob:none --quiet https://github.com/csuhan/timm_0_3_2.git /tmp/pip-req-build-ghq9_qab
  Resolved https://github.com/csuhan/timm_0_3_2.git to commit bf33f97f767a38e1b56cdd9c6a22372468fb55a4
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-jtyt1nh3
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-jtyt1nh3
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Using cached https://download.pytorch.org/whl/cu117/torch-2.0.0%2Bcu117-cp38-cp38-linux_x86_64.whl (1843.9 MB)
  Using cached https://download.pytorch.org/whl/cu117/torchvision-0.15.1%2Bcu117-cp38-cp38-linux_x86_64.whl (33.9 MB)
  U

In [ ]:
import os

path = './docs/cloth'
imgs_filenames = sorted(os.listdir(path)) # 파일 순으로 만들기
print(len(imgs_filenames))
print(imgs_filenames[:2])

imgs_dirs = []
for i in imgs_filenames:
    imgs_dirs.append(os.path.join(path, i))

print(len(imgs_dirs))
print(imgs_dirs[:2])

13679
['00000_00.jpg', '00001_00.jpg']
13679
['./docs/cloth/00000_00.jpg', './docs/cloth/00001_00.jpg']


In [ ]:
#demo
import cv2
import llama
import torch
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"

llama_dir = "/home/alpaco/namu/LLaMA-Adapter/llama_adapter_v2_multimodal7b/llama_model_weights/"

# choose from BIAS-7B, LORA-BIAS-7B, CAPTION-7B.pth
model, preprocess = llama.load("LORA-BIAS-7B", llama_dir, llama_type="7B", device=device)
model.eval()

prompt = llama.format_prompt('Just tell me two sentences for the color, occasions, style, type and fabric of A clothing based on this image.')
img = Image.fromarray(cv2.imread("/home/alpaco/namu/LLaMA-Adapter/llama_adapter_v2_multimodal7b/docs/cloth/00000_00.jpg"))
img = preprocess(img).unsqueeze(0).to(device)

result = model.generate(img, [prompt])[0]

print(result)

del prompt
del result
del img
del model
del preprocess

torch.cuda.empty_cache()
torch.cuda.memory_allocated()


Loading LLaMA-Adapter from ckpts/1bcbffc43484332672092e0024a8699a6eb5f558161aebf98a7c6b1db67224d1_LORA-BIAS-7B.pth
Trainable param: llama.layers.0.attention.wq.bias, torch.Size([4096]), torch.float32
Trainable param: llama.layers.0.attention.wo.bias, torch.Size([4096]), torch.float32
Trainable param: llama.layers.0.feed_forward.w1.bias, torch.Size([11008]), torch.float32
Trainable param: llama.layers.0.feed_forward.w2.bias, torch.Size([4096]), torch.float32
Trainable param: llama.layers.0.feed_forward.w3.bias, torch.Size([11008]), torch.float32
Trainable param: llama.layers.0.attention_norm.weight, torch.Size([4096]), torch.float32
Trainable param: llama.layers.0.ffn_norm.weight, torch.Size([4096]), torch.float32
Trainable param: llama.layers.1.attention.wq.bias, torch.Size([4096]), torch.float32
Trainable param: llama.layers.1.attention.wo.bias, torch.Size([4096]), torch.float32
Trainable param: llama.layers.1.feed_forward.w1.bias, torch.Size([11008]), torch.float32
Trainable param: l

8519680

In [ ]:
## 데이터 프레임만들기
import cv2
import llama
import torch
from PIL import Image
import os
import cv2
import pandas as pd
from PIL import Image
from pathlib import Path


device = "cuda" if torch.cuda.is_available() else "cpu"

llama_dir = "/home/alpaco/nsm/LLaMA-Adapter/llama_adapter_v2_multimodal7b/llama_model_weights/"

# choose from BIAS-7B, LORA-BIAS-7B, LORA-BIAS-7B-v21
model, preprocess = llama.load("LORA-BIAS-7B-v21", llama_dir, llama_type="7B", device=device)
model.eval()

# 결과를 저장할 CSV 파일 경로
csv_file_path = "/home/alpaco/nsm/LLaMA-Adapter/llama_adapter_v2_multimodal7b/output_final2-2.csv"


# 데이터프레임 초기화
df = pd.DataFrame()

from tqdm import tqdm

# 상위 100개 이미지에 대한 텍스트 생성 및 데이터프레임에 추가
for img_dir in tqdm(imgs_dirs[4000:8000], desc="이미지 처리 진행 중"):
    q = 'Just tell two sentences for the color, occasions, style, type and fabric of A clothing based on this image.'
    prompt = llama.format_prompt(q)
    img = Image.fromarray(cv2.imread(img_dir))
    img = preprocess(img).unsqueeze(0).to(device)
    result = model.generate(img, [prompt])[0]
    df = pd.concat([df, pd.DataFrame({"img_path": img_dir, "text": [result]})], ignore_index=True)
    #print(result)
    del prompt
    del result
    del img

# 결과를 CSV 파일로 저장
df.to_csv(csv_file_path, index=False)

print("데이터프레임이 CSV 파일로 저장되었습니다.")

del model
del preprocess

torch.cuda.empty_cache()
torch.cuda.memory_allocated()

Loading LLaMA-Adapter from ckpts/d26d107eec32127ac86ef1997cf7169de1c56a59c539fc1258c6798b969e289c_LORA-BIAS-7B-v21.pth
Trainable param: llama.layers.0.attention.wq.bias, torch.Size([4096]), torch.float32
Trainable param: llama.layers.0.attention.wo.bias, torch.Size([4096]), torch.float32
Trainable param: llama.layers.0.feed_forward.w1.bias, torch.Size([11008]), torch.float32
Trainable param: llama.layers.0.feed_forward.w2.bias, torch.Size([4096]), torch.float32
Trainable param: llama.layers.0.feed_forward.w3.bias, torch.Size([11008]), torch.float32
Trainable param: llama.layers.0.attention_norm.weight, torch.Size([4096]), torch.float32
Trainable param: llama.layers.0.ffn_norm.weight, torch.Size([4096]), torch.float32
Trainable param: llama.layers.1.attention.wq.bias, torch.Size([4096]), torch.float32
Trainable param: llama.layers.1.attention.wo.bias, torch.Size([4096]), torch.float32
Trainable param: llama.layers.1.feed_forward.w1.bias, torch.Size([11008]), torch.float32
Trainable para

이미지 처리 진행 중:   0%|          | 1/4000 [00:05<6:12:40,  5.59s/it]

The clothing in the image is blue, and it appears to be a casual style, suitable for everyday wear. The fabric is likely a cotton or blend, as it appears to be a lightweight and comfortable material.


이미지 처리 진행 중:   0%|          | 2/4000 [00:09<5:17:55,  4.77s/it]

The color of the clothing is pink, and it is a blouse. The occasion for wearing this clothing is casual, and it is a style that can be worn during the day. The fabric of the clothing is silk.


...

이미지 처리 진행 중: 100%|█████████▉| 3999/4000 [5:01:40<00:04,  4.02s/it]

The color of the clothing is black, and it is an occasion-specific dress, likely for a formal event. The style is elegant and classic, and the fabric is a high-quality silk.


이미지 처리 진행 중: 100%|██████████| 4000/4000 [5:01:45<00:00,  4.53s/it]

The clothing in the image is predominantly white, with a black polka-dot pattern. It appears to be a blouse or t-shirt and is likely to be worn during casual or everyday occasions. The style is simple and classic, while the type is likely a cotton blend fabric.
데이터프레임이 CSV 파일로 저장되었습니다.


8519680